In [1]:
#create a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

#print spark version
print(spark.version)

3.5.0


In [2]:
#read CNAES data
df_cnaes = spark.read.csv('/home/jovyan/data/cnaes.csv',\
                        header=True, inferSchema=True,\
                            sep=';', encoding='ISO-8859-1')
#print the df_cnaes schema
df_cnaes.printSchema()

root
 |-- CNAE: integer (nullable = true)
 |-- DESCRICAO_CNAE: string (nullable = true)



In [3]:
#count the rows
df_cnaes.count()

1359

In [5]:
# show df_cnaes
df_cnaes.show(5, truncate=False)

+------+---------------------------------------------------------+
|CNAE  |DESCRICAO_CNAE                                           |
+------+---------------------------------------------------------+
|111301|Cultivo de arroz                                         |
|111302|Cultivo de milho                                         |
|111303|Cultivo de trigo                                         |
|111399|Cultivo de outros cereais não especificados anteriormente|
|112101|Cultivo de algodão herbáceo                              |
+------+---------------------------------------------------------+
only showing top 5 rows



In [35]:
#read dataset estabelecimentos
df_estabelecimentos1 = spark.read.csv('/home/jovyan/data/estabelecimentos-1.csv',\
                        header=True, inferSchema=True, sep=';',\
                        encoding = 'ISO-8859-1')

df_estabelecimentos2 = spark.read.csv('/home/jovyan/data/estabelecimentos-2.csv',\
                        header=True, inferSchema=True, sep=';',\
                        encoding = 'ISO-8859-1')

df_estabelecimentos3 = spark.read.csv('/home/jovyan/data/estabelecimentos-3.csv',\
                        header=True, inferSchema=True, sep=';',\
                        encoding = 'ISO-8859-1')

In [54]:
#union csv files estabelecimentos-1, 2 e 3
df_estabelecimentos = df_estabelecimentos1.union(df_estabelecimentos2).union(df_estabelecimentos3)
df_estabelecimentos.coalesce(1).write.format("csv")\
    .option("delimiter", ";")\
    .option("header", "true")\
    .save("/home/jovyan/data/estabelecimentos.csv")

In [37]:
df_estabelecimentos.printSchema()

root
 |-- CNPJ_BASICO: integer (nullable = true)
 |-- CNPJ_ORDEM: integer (nullable = true)
 |-- CNPJ_DV: integer (nullable = true)
 |-- IDENTIFICADOR_MATRIZ_FILIAL: integer (nullable = true)
 |-- NOME_FANTASIA: string (nullable = true)
 |-- SITUACAO_CADASTRAL: integer (nullable = true)
 |-- DATA_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- MOTIVO_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- NOME_DA_CIDADE_NO_EXTERIOR: string (nullable = true)
 |-- PAIS: integer (nullable = true)
 |-- DATA_INICIO_ATIVIDADE: integer (nullable = true)
 |-- CNAE_PRINCIPAL: integer (nullable = true)
 |-- CNAE_SECUNDARIA: string (nullable = true)
 |-- TIPO_LOGRADOURO: string (nullable = true)
 |-- LOGRADOURO: string (nullable = true)
 |-- NUMERO: string (nullable = true)
 |-- COMPLEMENTO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- MUNICIPIO: integer (nullable = true)
 |-- DDD_1: string (nullable = true)

In [38]:
#rename column 'CNAE_PRINCIPAL' to 'CNAE'
df_estabelecimentos = df_estabelecimentos.withColumnRenamed('CNAE_PRINCIPAL','CNAE')

In [39]:
df_estabelecimentos.printSchema()

root
 |-- CNPJ_BASICO: integer (nullable = true)
 |-- CNPJ_ORDEM: integer (nullable = true)
 |-- CNPJ_DV: integer (nullable = true)
 |-- IDENTIFICADOR_MATRIZ_FILIAL: integer (nullable = true)
 |-- NOME_FANTASIA: string (nullable = true)
 |-- SITUACAO_CADASTRAL: integer (nullable = true)
 |-- DATA_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- MOTIVO_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- NOME_DA_CIDADE_NO_EXTERIOR: string (nullable = true)
 |-- PAIS: integer (nullable = true)
 |-- DATA_INICIO_ATIVIDADE: integer (nullable = true)
 |-- CNAE: integer (nullable = true)
 |-- CNAE_SECUNDARIA: string (nullable = true)
 |-- TIPO_LOGRADOURO: string (nullable = true)
 |-- LOGRADOURO: string (nullable = true)
 |-- NUMERO: string (nullable = true)
 |-- COMPLEMENTO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- MUNICIPIO: integer (nullable = true)
 |-- DDD_1: string (nullable = true)
 |-- TEL_

In [40]:
df_estabelecimentos.show(5, truncate=False)

+-----------+----------+-------+---------------------------+-------------+------------------+-----------------------+-------------------------+--------------------------+----+---------------------+-------+---------------+---------------+-----------------+------+---------------------------------+---------+--------+---+---------+-----+--------+-----+--------+-------+--------+---------------------------+-----------------+----------------------+
|CNPJ_BASICO|CNPJ_ORDEM|CNPJ_DV|IDENTIFICADOR_MATRIZ_FILIAL|NOME_FANTASIA|SITUACAO_CADASTRAL|DATA_SITUACAO_CADASTRAL|MOTIVO_SITUACAO_CADASTRAL|NOME_DA_CIDADE_NO_EXTERIOR|PAIS|DATA_INICIO_ATIVIDADE|CNAE   |CNAE_SECUNDARIA|TIPO_LOGRADOURO|LOGRADOURO       |NUMERO|COMPLEMENTO                      |BAIRRO   |CEP     |UF |MUNICIPIO|DDD_1|TEL_1   |DDD_2|TEL_2   |DDD_FAX|FAX     |CORREIO_ELETRONICO         |SITUACAO_ESPECIAL|DATA_SITUACAO_ESPECIAL|
+-----------+----------+-------+---------------------------+-------------+------------------+---------------

In [41]:
#left join estabelecimentos
estabelecimento_cnaes = df_estabelecimentos.join(df_cnaes, on='CNAE', how='left')

In [42]:
estabelecimento_cnaes.show(5, truncate=False)

+-------+-----------+----------+-------+---------------------------+-------------+------------------+-----------------------+-------------------------+--------------------------+----+---------------------+---------------+---------------+-----------------+------+---------------------------------+---------+--------+---+---------+-----+--------+-----+--------+-------+--------+---------------------------+-----------------+----------------------+----------------------------------------------------------------------------------------------------------------------------+
|CNAE   |CNPJ_BASICO|CNPJ_ORDEM|CNPJ_DV|IDENTIFICADOR_MATRIZ_FILIAL|NOME_FANTASIA|SITUACAO_CADASTRAL|DATA_SITUACAO_CADASTRAL|MOTIVO_SITUACAO_CADASTRAL|NOME_DA_CIDADE_NO_EXTERIOR|PAIS|DATA_INICIO_ATIVIDADE|CNAE_SECUNDARIA|TIPO_LOGRADOURO|LOGRADOURO       |NUMERO|COMPLEMENTO                      |BAIRRO   |CEP     |UF |MUNICIPIO|DDD_1|TEL_1   |DDD_2|TEL_2   |DDD_FAX|FAX     |CORREIO_ELETRONICO         |SITUACAO_ESPECIAL|DATA_SI

In [43]:
estabelecimento_cnaes.count()

20996744

In [44]:
df_estabelecimentos.count()

20996744

#### Pergunta 1: Quantos estabelecimentos existem?

In [45]:
df_estabelecimentos.count()

20996744

In [46]:
df_estabelecimentos.distinct().count()

20996744

#### Pergunta 2: Na tabela de estabelecimentos, quantas colunas existem e quantas são identificadas pelo spark como números? Use inferScehema ao ler os arquivos

In [47]:
num_columns = len(df_estabelecimentos.columns)
print(num_columns)

30


In [48]:
df_estabelecimentos.printSchema()

root
 |-- CNPJ_BASICO: integer (nullable = true)
 |-- CNPJ_ORDEM: integer (nullable = true)
 |-- CNPJ_DV: integer (nullable = true)
 |-- IDENTIFICADOR_MATRIZ_FILIAL: integer (nullable = true)
 |-- NOME_FANTASIA: string (nullable = true)
 |-- SITUACAO_CADASTRAL: integer (nullable = true)
 |-- DATA_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- MOTIVO_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- NOME_DA_CIDADE_NO_EXTERIOR: string (nullable = true)
 |-- PAIS: integer (nullable = true)
 |-- DATA_INICIO_ATIVIDADE: integer (nullable = true)
 |-- CNAE: integer (nullable = true)
 |-- CNAE_SECUNDARIA: string (nullable = true)
 |-- TIPO_LOGRADOURO: string (nullable = true)
 |-- LOGRADOURO: string (nullable = true)
 |-- NUMERO: string (nullable = true)
 |-- COMPLEMENTO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- MUNICIPIO: integer (nullable = true)
 |-- DDD_1: string (nullable = true)
 |-- TEL_

In [49]:
#count columns using list comprehension
num_numeric_columns = len([col for col, dtype in df_estabelecimentos.dtypes if dtype in ('integer')])
print("Number of numeric columns:", num_numeric_columns)


Number of numeric columns: 12


#### Pegunta 3: Usando estabelecimentos_df.write.parquet("estabelecimentos.parquet"), compare o tamanho dos arquivos parquet com os arquivos CSV originais. A economia de espaço foi da ordem de?

In [56]:
import os

csv_size = os.path.getsize('/home/jovyan/data/estabelecimentos.csv/part-00000-19cf8ea9-9d03-4f1b-b407-9e1156e70fb2-c000.csv')
csv_size

3477650646